In [1]:
from torch_contour.torch_contour1 import area, perimeter, hausdorff_distance
import torch

polygons1 = torch.tensor([
    [[0, 0], [1, 0], [1, 1], [0, 1]],  # Square
    [[0, 0], [2, 0], [2, 1], [0, 1]]   # Rectangle
], dtype=torch.float32)  # Permute to shape (B, 2, N)
polygons2 = torch.tensor([
    [[0, 0], [1, 0], [1, 1], [0, 1]],  # Another Square
    [[0, 0], [2, 0], [2, 2], [0, 2]]   # Another Rectangle
], dtype=torch.float32)  # Permute to shape (B, 2, N)


area_ = area(polygons1)
perimeter_ = perimeter(polygons2)
hausdorff_dists = hausdorff_distance(polygons1, polygons2)
print(area_)
print(perimeter_)


tensor([1.0000, 2.5000])
tensor([4., 8.])


In [308]:
from torch import nn

In [310]:
k = 1e5
size = 256
eps = 1e-8
contour = torch.tensor([[0.4,0.4],
                  [0.4,0.9],
                  [0.9,0.9],
                  [0.9,0.4],])[None]

if (contour < 0).any() or (contour > 1).any():

    raise ValueError("Tensor values should be in the range [0, 1]")

b = contour.shape[0]
n = contour.shape[1]
mesh = (
    torch.unsqueeze(
        torch.stack(
            torch.meshgrid(torch.arange(size), torch.arange(size)),
            dim=-1,
        ).reshape(-1, 2),
        dim=1,
    )
    / size
)
mesh = mesh.unsqueeze(0).repeat(b,1,1,1)
torch.pi = torch.acos(torch.zeros(1)).item() * 2
if (contour < 0).any() or (contour > 1).any():
    raise ValueError("Tensor values should be in the range [0, 1]")
contour = torch.unsqueeze(contour, dim=1)
diff = - mesh + contour
x = torch.max(torch.exp(-1e-20*torch.sum(diff**2,-1).reshape(b,-1,n)), dim = -1)[0].reshape(b,size,size)
roll_diff = torch.roll(diff, -1, dims=2)
norm_diff = torch.linalg.vector_norm(diff, dim=3)
norm_roll = torch.linalg.vector_norm(roll_diff, dim=3)
scalar_product = torch.sum(diff * roll_diff, dim=3)
clip = scalar_product / (norm_diff * norm_roll)
angles = torch.max(torch.exp(-k*(-1 - clip)**2), axis = -1)[0]
out0 = angles.reshape(b, size, size)+ x 


In [24]:

from torch import nn
class Draw_contour(nn.Module):
    """This layer draws a contour

    ...

    Attributes
    ----------
    size: int
        the size of the output image
    k: float
        the control parameter to approximate the sign function

    Methods
    -------
    forward(contour)
        forward function that draws the contour
    """

    def __init__(self, size, k=1e5):
        """
        Parameters
        ----------
        size: int
            the size of the output image
        k: float
            the control parameter to approximate the dirac function

        """
        super().__init__()
        self.k = k
        self.size = size

        self.mesh = (
            torch.unsqueeze(
                torch.stack(
                    torch.meshgrid(torch.arange(self.size), torch.arange(self.size)),
                    dim=-1,
                ).reshape(-1, 2),
                dim=1,
            )
            / self.size
        )

    def forward(self, contour):
        """Return the contour drawn on an image of the given size .

        Raises
        ------
        ValueError
            If the values of the contour or not between 0 and 1.
        """

        b,n,_ = contour.shape
        mesh = self.mesh.unsqueeze(0).repeat(b, 1, 1, 1)
        torch.pi = torch.acos(torch.zeros(1)).item() * 2

        if (contour < 0).any() or (contour > 1).any():
            raise ValueError("Tensor values should be in the range [0, 1]")

        contour = torch.unsqueeze(contour, dim=1)
        diff = -mesh + contour
        roll_diff = torch.roll(diff, -1, dims=2)
        x = torch.max(torch.exp(-self.k * torch.sum(diff**2, -1).reshape(b, -1, n)), dim=-1)[0].reshape(
            b, self.size, self.size
        )
        roll_diff = torch.roll(diff, -1, dims=2)
        norm_diff = torch.linalg.vector_norm(diff, dim=3)
        norm_roll = torch.linalg.vector_norm(roll_diff, dim=3)
        scalar_product = torch.sum(diff * roll_diff, dim=3)
        clip = scalar_product / (norm_diff * norm_roll)
        angles = torch.max(torch.exp(-self.k * (-1 - clip) ** 2), axis=-1)[0]
        contour_drawn = angles.reshape(b, self.size, self.size) + x

        return contour_drawn[:,None,...]

In [46]:
from torch_contour.torch_contour import area, perimeter
import torch

polygons1 = torch.tensor([
    [[0, 0], [1, 0], [1, 1], [0, 1]],  # Square
    [[0, 0], [2, 0], [2, 1], [0, 1]]   # Rectangle
], dtype=torch.float32)  # Permute to shape (B, 2, N)
print(polygons1.shape)
polygons2 = torch.tensor([
    [[0, 0], [1, 0], [1, 1], [0, 1]],  # Another Square
    [[0, 0], [2, 0], [2, 2], [0, 2]]   # Another Rectangle
], dtype=torch.float32).permute(0, 2, 1)  # Permute to shape (B, 2, N)

print(polygons1.shape)
area_ = area(polygons1)
perimeter_ = perimeter(polygons2)
print(area_)
# hausdorff_dists = hausdorff_distance(polygons1, polygons2)

torch.Size([2, 4, 2])
torch.Size([2, 4, 2])
tensor([[0., 0.]])


In [ ]:
k = 1e8
# x = torch.ones((2,5))
x = torch.tensor([[0.4,0.4],
                  [0.4,0.9],
                  [0.9,0.9],
                  [0.9,0.4],]).T
# print(x.shape)
size = 512
mesh = (

        torch.stack(
            torch.meshgrid(torch.arange(size), torch.arange(size)),
            dim=0,
        ).reshape(2,-1)
    / size
)
y_reverse = torch.flip(mesh,dims=(1,))
y_reverse[-1,:] =  - y_reverse[-1,:]
diff = torch.diff(x, n=1, dim=-1,append=x[:,0,None]).T
num_f = torch.matmul(diff,y_reverse)
tmp = (x[:,None,:] - mesh[...,None])
tmp = torch.cat((tmp, tmp[:,:,0,None]),dim=-1)


new =  torch.sum(torch.sigmoid(k * tmp[:,:,1:] * tmp[:,:,:-1]).T,dim=-1)/2



x_new = torch.cat([x,x[:,0][...,None]],dim=-1)
determ = x_new[0,1:]* x_new[1,:-1] - x_new[0,:-1]* x_new[1,1:]
distances = torch.abs(num_f - determ[...,None])/(torch.norm(diff,dim = -1)[...,None] +1e-7)
projection_contour = torch.max(torch.exp(-50*distances),dim=0)[0]